#FashionMNIST Image Classification
The training of this Neural Network consists of 8 steps:


1.   Import the relevant packages.
2.   Build a dataset that can fetch data one data point at a time.
3.   Wrap the DataLoader from the dataset.
4.   Build a model and then define the loss function and the optimizer.
5.   Define two functions to train and validate a batch of data, respecitvely.
6.   Define a function that will calculate the accuracy of the data.
7.   Perform weight updates based on each batch of data over increasing epochs.
8.   Plot the variation of the training loss and accuracy. 



#1. Import the relevant packages.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch.optim import SGD, Adam
from torch import optim
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 2. Build a dataset that can fetch data one data point at a time.




In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
data_folder = "./data/FMNIST"
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

val_fmnist = datasets.FashionMNIST(data_folder, download=True, train=False)
val_images = val_fmnist.data
val_targets = val_fmnist.targets

Scaling a dataset is the process of ensuring that the variables are confined to a finte range. The reason for this is because the activation function makes it so that the only changes done are when the weight values are very small.

In [ ]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y):
        x = x.float() / 255
        x = x.view(-1, 28 * 28)  # Flatten Image to 1x784
        self.x, self.y = x, y

    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.to(device), y.to(device)

    def __len__(self):
        return len(self.x)

# 3. Wrap the DataLoader from the dataset
Create a function that generates a training DataLoader.

In [ ]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    val = FMNISTDataset(val_images, val_targets)
    val_dl = Dataloader(val, batch_size = len(val_images), shuffle=False)
    return trn_dl, val_dl

# 4. Build a model and then define the loss function and the optimizer.
Define a model, as well as the loss function and the optimizer.

The model is a network with one hidden layer containing 1,000 neurons. The output is a 10-neuron layer since there are 10 possible classes. Furthermore, we are calling the CrossEntropyLoss function since the output can belong to any of the 10 classes for each image. 


In [ ]:
def get_model():
  class neuralnet(nn.Module):
    def __init__(self):
      super().__init__()
      self.input_to_hidden_layer = nn.Linear(784, 1000)
      self.dropout = nn.Dropout(0.25)
      self.batch_norm = nn.BatchNorm1d(1000)
      self.hidden_layer_activation = nn.ReLU()
      self.hidden_to_output_layer == nn.Linear(1000, 10)
    
    def forward(self, x):
      x = self.dropout(x)
      x = self.input_to_hidden_layer(x)
      x0 = self.batch_norm(x)
      x1 = self.hidden_layer_activation(x0)
      x1 = self.dropout(x1)
      x2 = self.hidden_to_output_layer(x1)

      return x2, x1
  
  model = neuralnet().to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = Adam(model.parameters(), lr=1e-3)
  return model, loss_fn, optimizer

# 5. Define two functions to train and validate a batch of data, respectively.
Define a function that will train the dataset on a batch of images:

In [ ]:
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()
    prediction = model(x)[0]
    batch_loss = loss_fn(prediction, y)  # Compute Loss
    batch_loss.backward()  # Computes all the gradients of 'model.parameters()'
    optimizer.step()  # Apply new-weights = f(old-weights, old-weight-gradients) | f <- Optimizer
    optimizer.zero_grad()  # Flush gradients memory for next batch of calculation
    return batch_loss.item()

# 6. Define a function that will calculate teh accuracy of the data.
Build a function that calculates the accuracy of a given dataset.

In [ ]:
def accuracy(x, y, model):
    model.eval()
    with torch.no_grad():
      prediction = model(x)[0]
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

#7. Perform weight updates based on each batch of data over increasing epochs. 
Train the neural network usin gthe following lines of code.

In [ ]:
trn_dl = get_data()
model, loss_fn, optimizer = get_model()
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                 factor=0.5, patience=0,
                                                 threshold=0.001,
                                                 verbose=True,
                                                 min_lr=1e-5,
                                                 threshold_mode='abs')
for epoch in range(30):
    print(epoch)
    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
      x, y = batch
      val_is_correct = accuracy(x, y, model)
      validation_loss = val_loss(x, y, model)
      scheduler.step(validation_loss)
      val_epoch_accuracy = np.mean(val_is_correct)

      train_losses.append(train_epoch_loss)
      train_accuracies.append(train_epoch_accuracy)
      val_losses.append(validation_loss)
      val_accuracies.append(val_epoch_accuracy)

# 8. Plot the variation of the training loss and accuracy. 
The variation of the training loss and accuracy over increasing epochs can be displayed using the following code:

In [ ]:
epochs = np.arange(5) + 1
plt.figure(figsize=(20, 5))
plt.subplot(121)
plt.title('Loss value over increasing epochs')
plt.plot(epochs, losses, label='Training Accuracy')
plt.legend()
plt.subplot(122)
plt.title('Accuracy value over increasing epochs')
plt.plot(epochs, accuracies, label='Training Accuracy')
plt.gca().set_yticklabels(['{.0f}%'.format(x * 100) for x in plt.gca().get_yticks()])
plt.legend()